## Timothy Miller
## GTECH 73100, Dr. Sun

# [Assignment Eight](https://github.com/TangoYankee/gtech_731-geocomp-hw/tree/main/assignment-eight)
States and counties in geopandas

### Import modules

In [24]:
import pandas as pd
import geopandas as gpd
import json
import io

### Translate encoding of json file
The original "data/gz_2010_us_050_00_20m.json" file appears to be encoded in 'latin-1', causing errors when attempting read it directly into geopandas
This is resolved by first reading in the file as json and then reexporting as json, changing the name to geojson

In [11]:
with io.open("data/gz_2010_us_050_00_20m.json", encoding='latin-1') as f:
    data = json.load(f)

with open('data/gz_2010_us_050_00_20m.geojson', 'w') as fp:
    json.dump(data, fp)

### Read files 

In [111]:
county_data = gpd.read_file("data/gz_2010_us_050_00_20m.geojson")

with io.open("data/fipsToState.json") as f:
    fips_to_state = json.load(f)

with io.open("data/stateCodeToFips.json") as f:
    state_code_to_fips = json.load(f)

In [112]:
fips_to_state_df = pd.DataFrame({ 
    "fips": fips_to_state.keys(),
    "state": fips_to_state.values(),
 })

state_code_to_fips_df = pd.DataFrame({
    "state_code": state_code_to_fips.keys(),
    "fips": state_code_to_fips.values()
})

### Task 1
Find the top n most common county names